In [18]:
import pandas as pd
import numpy as np

# [Step 1] 원본 데이터 로드 및 복사
customer = pd.read_csv("customer_hm.csv")
transactions = pd.read_csv("transactions_hm.csv")
articles = pd.read_csv("articles_hm.csv")

# 경로에 맞게 위 아래 중 선택 주석처리
# customer = pd.read_csv('../../data/customer_hm.csv')
# transactions = pd.read_csv('../../data/transactions_hm.csv')
# articles = pd.read_csv('../../data/articles_hm.csv')

#copy
df_cust = customer.copy()
df_tran = transactions.copy()
df_art = articles.copy()

# [Customer] 데이터 전처리

#1. fashion_news_frequency가 결측치인 데이터만 추출 (결측치 Regularly로 대체)
missing_data = df_cust[df_cust['fashion_news_frequency'].isnull()]

#1-2. 결측치 'Regularly'으로 채우기
df_cust['fashion_news_frequency'] = df_cust['fashion_news_frequency'].fillna('Regularly')

#2. 문자형 정수형으로 매핑 (사전 정의)
status_map = {'ACTIVE': 2, 'PRE-CREATE': 1, 'LEFT CLUB': 0}
frequency_map = {'Regularly': 2, 'Monthly': 1, 'NONE': 0}

#2-2.  매핑 적용
df_cust['club_member_status'] = df_cust['club_member_status'].map(status_map)
df_cust['fashion_news_frequency'] = df_cust['fashion_news_frequency'].map(frequency_map)


#3. 연령대 그룹화 (age_segment) : 나이를 나이대별로 나눠 파생변수 생성
def cate_age(age):
    if age < 20: return '10대'
    elif age < 30: return '20대'
    elif age < 40: return '30대'
    elif age < 50: return '40대'
    elif age < 60: return'50대'
    else: return '60대 이상'

df_cust['age_segment'] = df_cust['age'].apply(cate_age)


# [Transactions] 데이터 전처리

#1. join을 위해 동일한 타입으로 변환
df_tran['customer_id']= df_tran['customer_id'].astype(str)
df_cust['customer_id']= df_cust['customer_id'].astype(str)

df_tran['article_id']= df_tran['article_id'].astype(str).str.zfill(10)
df_art['article_id'] = df_art['article_id'].astype(str).str.zfill(10)

#2. 데이터 크기 및 중복 제거
df_tran.drop_duplicates(inplace=True)

#3. 날짜 타입 변환 및 월 컬럼 생성
df_tran['t_dat'] = pd.to_datetime(df_tran['t_dat'], format='%Y-%m-%d')
df_tran['year_month'] = df_tran['t_dat'].dt.to_period('M')

#4. 판매 채널 한글 컬럼 생성 및 값 변환
df_tran["channel"] = df_tran["sales_channel_id"].map({1: "오프라인", 2: "온라인"})

df_tran["channel"].value_counts() 
numbs = df_tran["channel"].value_counts()
print("온라인:", f"{numbs['온라인']:,}")
print("오프라인:", f"{numbs['오프라인']:,}")
print("전체 거래 건수:", f"{df_tran.shape[0]:,}")

#가격 데이터 분포 확인
df_tran['price'].describe() 
print("price가 0.1 이상:", len(df_tran[df_tran["price"] >= 0.1]))
print("price가 0.2 이상:", len(df_tran[df_tran["price"] >= 0.2]))
print("price가 0.3 이상:", len(df_tran[df_tran["price"] >= 0.3]))
print("price가 0.4 이상:", len(df_tran[df_tran["price"] >= 0.4]))
print("price가 0.5 이상:", len(df_tran[df_tran["price"] >= 0.5]))

#고가 거래 분리 및 채널 확인
high_price_transactions = df_tran[df_tran["price"] >= 0.4]
high_price_transactions['sales_channel_id'].replace({1: 'Offline', 2: 'Online'}).value_counts() 

#월별 거래 건수 & 매출액 계산
monthly_sales = df_tran.groupby('year_month').size()    #행=거래량
monthly_price = df_tran.groupby('year_month')['price'].sum()    #매출액뽑기

print(monthly_price)

#최고 매출 월 
best_month = monthly_price.idxmax()
best_value = monthly_price.max()

print("가장 매출이 높은 달:", best_month)
print("그 달의 매출:", best_value) 


# [Article] 데이터 전처리: 상품 전략 파생변수 생성

#1. ID 형식 통일
df_art['article_id'] = df_art['article_id'].astype(str).str.zfill(10)

#2. 결측치(NULL) 제어 (detail_desc의 NULL값을 No Description로)
df_art['detail_desc'] = df_art['detail_desc'].fillna('No Description')

#3. 불필요 컬럼 제거 (메모리 줄이기) : Name(이름)이 존재하여 중복 정보인 Code(코드성 숫자) 컬럼 불필요
cols_to_drop = [
    'product_type_no', 
    'graphical_appearance_no', 
    'colour_group_code', 
    'perceived_colour_value_id',
    'perceived_colour_master_id',
    'department_no',
    'index_code',
    'index_group_no',
    'section_no',
    'garment_group_no'
]
df_art_cleaned = df_art.drop(columns=cols_to_drop) 

#4. 매핑 및 파생변수 설정
#4-1. 시즌 별 매핑 (product_season) / 시즌성 구분을 위한 키워드 정의
def get_season(row):
    # section_name과 prod_name을 합쳐서 판단
    text = (str(row['section_name']) + " " + str(row['prod_name'])).lower()
    group = str(row['product_group_name']).lower()
    
    # 1. SS
    ss_keywords = [
        'swimwear', 'sport', 'shorts', 'sandals', 'sleeveless', 
        'tank', 'mini', 'beach', 'summer', 'sun', 'flip flop', 
        'skirt', 'dress'
    ]

    # 2. FW 
    fw_keywords = [
        'outerwear', 'knitted', 
        'jacket', 'coat', 'hoodie', 'sweatshirt', 'scarf', 'gloves', 
        'boots', 'heavy', 'fur', 'wool', 'cardigan', 'winter'
    ]
    
    if any(kw in text for kw in ss_keywords) or group in ['swimwear']:
        return 'SS'
    # FW는 오직 두꺼운 옷 키워드가 있을 때만! (items of clothing 조건 제거)
    elif any(kw in text for kw in fw_keywords):
        return 'FW'
    else:
        return 'All-Season'

#4-2. 파생변수 생성 및 axis=1로 왼쪽에서 오른쪽으로(행 방향) 읽도록 조정
df_art_cleaned['product_season'] = df_art_cleaned.apply(get_season, axis=1)
#4-3. 메인 카테고리 정의 (원본 보존)
df_art_cleaned['category_main'] = df_art_cleaned['index_group_name'] 


#5. 저관여/고관여 제품 분류 (product_strategy)
def get_involvement_strategy(row):
    garment = str(row['garment_group_name']).lower()
    section = str(row['section_name']).lower()
    # 1. 저관여 (베이직)
    if any(kw in garment for kw in ['basic', 'underwear', 'socks', 'jersey']) or \
       'basic' in section:
        return 'Low_Involvement_Basic'
    # 2. 고관여 (전략템/트렌드템)
    elif any(kw in garment for kw in ['knitwear', 'outerwear', 'dresses']) or \
         any(kw in section for kw in ['trend', 'special']):
        return 'High_Involvement_Strategic'
    else:
        return 'General_Fashion'
    
df_art_cleaned['product_strategy'] = df_art_cleaned.apply(get_involvement_strategy, axis=1)

#6. 신상품/트렌드 키워드 추출 (is_new)
def check_newness(row):
    text = (str(row['prod_name']) + " " + str(row['detail_desc'])).lower()
    new_keywords = ['new', 'collection', 'latest', 'trend', 'exclusive']
    
    if any(kw in text for kw in new_keywords):
        return 'New_Arrival'
    return 'Regular_Carryover'

df_art_cleaned['is_new'] = df_art_cleaned.apply(check_newness, axis=1)

#7. 색상 그룹화 (color_tone)
def get_color_tone(color):
    color = str(color).lower()
    dark_colors = ['black', 'dark blue', 'dark grey', 'dark red', 'dark pink', 'dark purple', 'dark green', 'dark orange', 'dark beige', 'dark turquoise', 'dark yellow', 'grey', 'bronze/copper']
    light_colors = ['white', 'off white', 'light beige', 'light blue', 'light grey', 'light orange', 'light pink', 'light green', 'light red', 'light yellow', 'light purple', 'transparent', 'light turquoise']
    
    if any(dc in color for dc in dark_colors):
        return 'Dark_Tone'
    elif any(lc in color for lc in light_colors):
        return 'Light_Tone'
    else:
        return 'Neutral_Tone'

df_art_cleaned['color_tone'] = df_art_cleaned['colour_group_name'].apply(get_color_tone)

온라인: 721,488
오프라인: 318,613
전체 거래 건수: 1,040,101
price가 0.1 이상: 10704
price가 0.2 이상: 719
price가 0.3 이상: 126
price가 0.4 이상: 29
price가 0.5 이상: 2
year_month
2019-01    2129.926131
2019-02    1989.217641
2019-03    2374.905504
2019-04    2703.443538
2019-05    2748.199469
2019-06    3088.776976
2019-07    2552.035334
2019-08    1943.422489
2019-09    2559.226862
2019-10    2358.486793
2019-11    2463.769270
2019-12    1985.492149
Freq: M, Name: price, dtype: float64
가장 매출이 높은 달: 2019-06
그 달의 매출: 3088.776976142


In [19]:
##체크 용도
#df_cust
display(df_cust[['customer_id', 'age', 'age_segment', 'club_member_status', 'fashion_news_frequency']].head())

#df_tran
display(df_tran[['customer_id', 'article_id', 't_dat', 'year_month', 'channel', 'price']].head())

#df_art_cleaned
display(df_art_cleaned[['article_id', 'prod_name', 'product_strategy', 'is_new', 'product_season', 'color_tone']].head())

,customer_id,age,age_segment,club_member_status,fashion_news_frequency
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,49,40대,2,0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,25,20대,2,0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24,20대,2,0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,54,50대,2,0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,52,50대,2,2


,customer_id,article_id,t_dat,year_month,channel,price
0,3e2b60b679e62fb49516105b975560082922011dd752ec...,0698328010,2019-11-05,2019-11,온라인,0.016932
1,89647ac2274f54c770aaa4b326e0eea09610c252381f37...,0760597002,2019-05-22,2019-05,온라인,0.033881
2,2ebe392150feb60ca89caa8eff6c08b7ef1138cd6fdc71...,0488561032,2019-05-10,2019-05,온라인,0.016932
3,7b3205de4ca17a339624eb5e3086698e9984eba6b47c56...,0682771001,2019-08-26,2019-08,온라인,0.033881
4,3b77905de8b32045f08cedb79200cdfa477e9562429a39...,0742400033,2019-08-10,2019-08,오프라인,0.003220


,article_id,prod_name,product_strategy,is_new,product_season,color_tone
0,0108775015,Strap top,Low_Involvement_Basic,Regular_Carryover,All-Season,Dark_Tone
1,0108775044,Strap top,Low_Involvement_Basic,Regular_Carryover,All-Season,Light_Tone
2,0108775051,Strap top (1),Low_Involvement_Basic,Regular_Carryover,All-Season,Light_Tone
3,0110065001,OP T-shirt (Idro),General_Fashion,Regular_Carryover,All-Season,Dark_Tone
4,0110065002,OP T-shirt (Idro),General_Fashion,Regular_Carryover,All-Season,Light_Tone


### [Data Dictionary] 

#### 1. 파생변수
- **product_strategy** : 상품 관여도 전략 (Basic / Strategic / General)
- **age_segment** : 고객 연령대 세그먼트 (10대 ~ 60대+)
- **is_new** : 신상품 반응 지표 (New_Arrival / Regular_Carryover)
- **product_season** : 상품 시즌 속성 (FW / SS / All-Season)
- **color_tone** : 디자인 톤앤매너 (Dark / Light / Neutral)

#### 2. 기본 정보 (Raw Data)
- **channel** : 온/오프라인 구매 채널 (매출 발생 지점 확인)
- **club_member_status** : 멤버십 등급 (로열티 고객의 구매 유지력 확인)
- **fashion_news_freq** : 마케팅 뉴스레터 구독 빈도 (CRM 활동 영향력 측정)
- **price** : 상품 판매 단가 (객단가 및 가격대별 수요 분석)
- **year_month** : 시간 지표 (6월 정점 대비 8월 하락세의 기준축)
- **category_main** : 여성/남성/아동 등 메인 카테고리 (품목별 성과 측정)

In [20]:
# [JOIN] df_tran 기준 Left Join
# 고객의 나이대(age_segment), 뉴스 구독 빈도 등을 거래 내역에 연결
df_merged = pd.merge(
    df_tran, 
    df_cust, 
    on='customer_id', 
    how='left'
)

print("결합 후 데이터 크기:", df_merged.shape)
print(df_merged.head())

결합 후 데이터 크기: (1040101, 13)
       t_dat                                        customer_id  article_id  \
0 2019-11-05  3e2b60b679e62fb49516105b975560082922011dd752ec...  0698328010   
1 2019-05-22  89647ac2274f54c770aaa4b326e0eea09610c252381f37...  0760597002   
2 2019-05-10  2ebe392150feb60ca89caa8eff6c08b7ef1138cd6fdc71...  0488561032   
3 2019-08-26  7b3205de4ca17a339624eb5e3086698e9984eba6b47c56...  0682771001   
4 2019-08-10  3b77905de8b32045f08cedb79200cdfa477e9562429a39...  0742400033   

      price  sales_channel_id year_month channel   FN  Active  \
0  0.016932                 2    2019-11     온라인  1.0     1.0   
1  0.033881                 2    2019-05     온라인  0.0     0.0   
2  0.016932                 2    2019-05     온라인  1.0     1.0   
3  0.033881                 2    2019-08     온라인  0.0     0.0   
4  0.003220                 1    2019-08    오프라인  0.0     0.0   

   club_member_status  fashion_news_frequency   age age_segment  
0                 2.0                    

In [21]:
# [JOIN] df_final
df_final = pd.merge(
    df_merged, 
    df_art_cleaned, 
    on='article_id', 
    how='left'
)

In [22]:
import os
print(os.getcwd())
print(os.listdir('data'))

d:\sonamu\project_01\SOL_analysis
['cleaned_data.csv']


In [23]:
#df_final.to_csv('data/cleaned_data.csv', index=False)
df_final.to_csv(r'D:\sonamu\project_01\SOL_analysis\data\cleaned_data.csv', index=False)

#불러올 때 :
# df = pd.read_csv('data/cleaned_data.csv')

In [24]:
# ⚓4-1. 시즌 별 매핑 (product_season) / 시즌성 구분을 위한 키워드 정의
def get_season(row):
    """
    상품의 이름과 소속 섹션을 훑어 '여름(SS)', '겨울(FW)', '사계절(All-Season)' 이름표를 붙여주는 함수입니다.
    """
    # 🗣️ 항해사의 한마디: 섹션명과 제품명을 합쳐 텍스트라는 거대한 바다를 만들고, 
    # 대소문자 구분을 없애(lower) 'Swim'이든 'swim'이든 다 잡아내도록 설계했습니다. [cite: 2026-01-22]
    text = (str(row['section_name']) + " " + str(row['prod_name'])).lower()
    group = str(row['product_group_name']).lower()
    
    # 🌊 1. SS(Spring/Summer): 여름을 상징하는 가벼운 보물 키워드
    ss_keywords = [
        'swimwear', 'sport', 'shorts', 'sandals', 'sleeveless', 
        'tank', 'mini', 'beach', 'summer', 'sun', 'flip flop', 
        'skirt', 'dress'
    ]

    # ❄️ 2. FW(Fall/Winter): 겨울을 견디는 두껍고 묵직한 보물 키워드
    fw_keywords = [
        'outerwear', 'knitted', 
        'jacket', 'coat', 'hoodie', 'sweatshirt', 'scarf', 'gloves', 
        'boots', 'heavy', 'fur', 'wool', 'cardigan', 'winter'
    ]
    
    # 🎯 1순위 판별: 여름 키워드가 하나라도 있거나, 아예 수영복 그룹인 경우 'SS' 항구로 예인합니다.
    if any(kw in text for kw in ss_keywords) or group in ['swimwear']:
        return 'SS'
    
    # 🎯 2순위 판별: 여름 키워드는 없으면서 두꺼운 옷 키워드가 있다면 'FW' 항구로 보냅니다.
    elif any(kw in text for kw in fw_keywords):
        return 'FW'
    
    # 🎯 최종 판별: 위의 그물에 걸리지 않은 면티, 양말 등 베이직 아이템은 'All-Season'으로 분류합니다.
    else:
        return 'All-Season'

# ⚓4-2. 파생변수 생성: 함수를 모든 행(axis=1)에 적용하여 'product_season' 컬럼을 생성합니다.
# 🗣️ 항해사의 한마디: apply(axis=1)은 항해사가 각 방(Row)을 돌며 상품을 하나씩 검사하는 것과 같습니다. [cite: 2026-01-21]
df_art_cleaned['product_season'] = df_art_cleaned.apply(get_season, axis=1)

# ⚓4-3. 메인 카테고리 정의 (원본 보존)
# 'index_group_name'(여성복, 남성복 등)을 'category_main'이라는 직관적인 이름으로 복사합니다.
df_art_cleaned['category_main'] = df_art_cleaned['index_group_name']